### CV Practical Guide (Lucas-Kanade for Optical Flow)

In [4]:
import cv2
import numpy as np

In [5]:
video = cv2.VideoCapture("C:/Users/cw/Downloads/video_circular_cars.mp4")

In [6]:
lk_params = dict(winSize=(75, 75),                               #Lucas-Kanade parameters
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [7]:
color = np.random.randint(0, 255, (100, 3))

In [8]:
ret, prev_frame = video.read()                                                  # frame grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

In [9]:
print("Frame size:", prev_frame.shape)

Frame size: (1920, 1080, 3)


In [10]:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)     # feature extraction from first frame 
prev_corners = cv2.goodFeaturesToTrack(prev_gray, maxCorners=500, qualityLevel=0.3, minDistance=7, blockSize=7, mask=None)

In [11]:
mask = np.zeros_like(prev_frame)
print("Mask size:", mask.shape)

Mask size: (1920, 1080, 3)


In [ ]:
 # Process the video frame by frame
 while True:
    ret, curr_frame = video.read()
    if not ret:
        break
        
    # current frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
     
    # Calculate optical flow using Lucas-Kanade method
    curr_corners, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)
    # Filters valid points
    good_old = prev_corners[status == 1]
    good_new = curr_corners[status == 1]
    # Draw motion vectors
    for (old, new) in zip(good_old, good_new):
        x_old, y_old = old.ravel()
        x_new, y_new = new.ravel()
        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 0, 255), 2)
        curr_frame = cv2.circle(curr_frame, (int(x_new), int(y_new)), 5, (0, 255, 0), -1)
    # Combine the frame and the mask for visualization
    output = cv2.add(curr_frame, mask)
    # Display the result
    cv2.namedWindow("Lucas-Kanade Optical Flow", cv2.WINDOW_NORMAL)  # Make the display window resizable
    cv2.imshow("Lucas-Kanade Optical Flow", output)

    # Break the Loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    prev_gray = curr_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)

In [ ]:
video.release()
cv2.destroyAllWindows()